In [ ]:
# default_exp core

# Implementing autograd from scratch

> This will have both code and documentation in it

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace
from IPython.display import Markdown as md


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np
import autograd 
path_assets = './assets/'

### Prerequsites 

* you know what backpropagation is and roughly how it works
* you know what autodifferentiation is 

### What are we trying to find?


We have to have a function we want to differentiate. This function will take in some number of variables: one, two, three... however many. If our function is called $f$, we are looking to find $df/dv$, where $v$ is one of these variables.  

Here we define a function in two variables: x and y. $$f = \log(x^2) + y^2 + xy$$

In [ ]:
def f(x,y):    return np.log(x**2) + y**2 + x*y

The aim is to create a function `grad` that takes `f` as an input and returns a function with its gradient wrt a variable. So for our function above, `grad` can give us the answer of either $df/dx$ or $df/dy$, depending on what variable we tell it to differentiate with respect to. Let's define `grad`, giving it dummy values for now.

How do we choose this variable? We don't do it by name, telling the function either 'x' or 'y'. Rather, we give it a number (call it `argnum`), and say, "Differentiate `f` with respect to the `argnum`th input." `f` has an argument list, and `argnum` is the index of this variable list, which corresponds to the variable we are interested in. 

`grad` has a nested structure: it returns a function `gradfun` that in turn returns the gradient. Notice the use of `*args` and `**kwargs` below. This is needed in the inner function `gradfun` because a) it doesnt' know how many arguments `f` takes; b) you can choose to call those arguments either with or without keywords. If you call them without keywords, the arguments are stored in `*args`, and if you call them with keywords, the arguments are stores in `**kwargs`.

In [ ]:
def grad(f, argnum = 0): 
    """Returns a function that finds the gradient"""
    def gradfun(*args, **kwargs):
        """Returns the actual gradient """
        #set_trace()
        print ("Args", args)
        print ("Kwargs", kwargs)
        # Dummy values. Returns correct gradient only for our function f above. 
        # Use these values until we calculate the true ones using autodiff. 
        #### remove this code once true code written
        if   argnum == 0: return 2*x * np.log(x**2) + y  # df/dx
        elif argnum == 1: return 2*y + x                 # df/dy
        #### 
        # true autograd code goes here 
        ####
    return gradfun
# example usage
dfdx = grad(f, argnum = 0)
dfdy = grad(f, argnum = 1)
print("dfdx", dfdx(1,2))      # call gradient w/out keywords, values go into *args    in gradfun 
print("dfdy", dfdy(x=13,y=4)) # call gradient with  keywords, values go into **kwargs in gradfun 

Args (1, 2)
Kwargs {}
dfdx 137.37736658799992
Args ()
Kwargs {'x': 13, 'y': 4}
dfdy 21


So that's our goal. We are trying to build this `grad` function properly, following the structure defined above. 

### Building a computation graph 

Say you had some expression, like $ (4 \times 5) + 2 - 4$. I'm sure you know the answer to this, but how would a computer work it out? 

You may remember the order of operations used to work out these expressions; I learnt the acronym [BODMAS](https://www.mathsisfun.com/operation-order-bodmas.html) to remember these. Python has its own order of operations too, governed by the hierarchy of [operator precedence](https://docs.python.org/3/reference/expressions.html#operator-precedence). Python breaks down an expression and executes it in this order. You can view the breakdown of an expression as a computation graph, where each node is an action (x3, +5, log) and where the graph doesn't have any loops (it is a DAG). 

The computation graph is very important to us. It is used in the backpropagation algorithm. The idea is that becuase each node of the graph is a simple operation (like +, x, log), working out its derivative is also easy. The gradient at a point on the computation graph is called the local gradient. Start at the head node and combine local gradients together until you reach a leaf node. Then this leaf node has the answer you seek. 

Now we need a way to build the computation graph. This is quite a clever idea actually. Typically your functions are made up of operators from the numpy package. What we will do is create a copy of the numpy package and use that copy instead of the original numpy package. The copy works exactly like the original does, except that when it executes functions, it builds a computation graph for us. 

The easiest way to show this is by an example. Let's say we had the following function: 
$$ logistic(z) = \frac{1}{1 + \exp(-z)} $$

We would implement this in code like this

In [ ]:
def logistic(z): return 1 / (1 + np.exp(-z))


Numpy typically overwrites common operators, meaning it replaces $+, \times, / $ with its own numpy equivalents `np.add`, `np.multiply`, `np.divide` and so on. It does this defining the methods  `__add__`,`__mult__`,`__div__` in the `numpy.ndarray` class. So the effect of this is that `logistic(z)` gets transformed into something like this: 

In [ ]:
def logistic2(z): return np.reciprocal(np.add(1, np.exp(np.negative(z))))

Let's apply the order of operations, or in other words the order that Python breaks down this expression. This has the same effect as constructing a number of intermediate variables, one after each operation. Call the intermediate variables $t_1, t_2, t_3...$, the input to the function $z$, and the final value $y$. 

In [ ]:
def logistic3(z): 
    t1 = np.negative(z) 
    t2 = np.exp(t1)
    t3 = np.add(1, t2)
    y = np.reciprocal(t3) 
    return y

Next we want to turn `logistic2` into a computation graph, with nodes and links between them. Here's what this graph looks like.

<img src="assets/node_tree.svg" width="240"/>

We will eventually define a class called `Node` and each node in the graph will be a member of this class. We define the links between nodes with the 'parents' attribute. Leaf nodes do not have parents. The leaf nodes above are $1$ and $z$.  

Below is a representation of the computational graphs, now using Nodes. The numbers in `value` indicate the value of that intermediate variable. The function was given $z=1.5$ as an input and outputs $y=0.818$. 

<img src="assets/node_tree.png" width="800"/>

Let's confirm we get the same answer. 

In [ ]:
np.round(logistic3(1.5),3)

0.818

#### Constructing the Node class

Now we can construct a first version of the Node class. For each Node, we need at least `value`, a function (`fun`) and `parents`. Let's create an tuple called `recipe` that we store `fun` and `value` in. 

We also will create a function called `initialise_root` that starts off the graph. A root of the tree doesn't have any parents, its function is just the identity function, and it has no value. 

In [ ]:
class Node:
    """A node in a computation graph."""
    def __init__(self, value, fun, parents):
        self.parents = parents
        self.recipe = (fun, value)

    def initialize_root(self):
        self.parents = []
        self.recipe = (lambda x: x, None)

Now we have the Node class, we could manually build a computational graph if we wanted to. (We don't create a Node for $1$ or other scalars, just intermediate variables)

In [ ]:
val_z = 1.5 
z = Node(val_z, None, [])
val_t1 = np.negative(val_z)
t1 = Node(val_t1,np.negative, [z])
val_t2 = np.exp(val_t1)
t2 = Node(val_t2, np.exp, [t1])
val_t3 = np.add(val_t2, 1)
t3 = Node(val_t3, np.add, [t2])
val_y = np.reciprocal(val_t3)
y = Node(val_y, np.reciprocal, [t3])
print(round(y.recipe[1],3)) # same answer as before

0.818


Creating the computational graph this way is both manual and clunky. Time now to build it automatically. 

### Creating a new version of Numpy

It would be great if numpy created a node for each intermediate variable and added it to our graph. But it won't do that. So it's time to make our own version of Numpy so we can trace the flow of execution and create our computation graph. 

First we'll delete the reference to our current numpy and import it under a new name, `onp`

In [ ]:
try:     del np
except:  pass 
import numpy as onp

Throughout this section, I'll refer to the original numpy as `onp`, and the version we are building as `anp`. 

There's a few things we have to do: 

* create new versions of `onp` functions. For example, we need to create a function `anp.add` that does everything `onp.add` does, but it also adds a node to the computation graph when called. Same with `anp.multiply`, `anp.divide` etc. 
* overload operators like $+, \times, /$ so that they use the `anp` versions: `anp.add`, `anp.multiply`, `anp.divide`, by defining functions like `__add__`, `__mul__`, `__div__`. 


#### Primitives 

#### Boxes 

### Wrapping Nodes in Boxes 

Boxes are used to indicate the variable you are differentiating with respect to 

### Making VJPs

### Backwards pass

### Resources

I used the following resources to put this document together. 

Lecture slides by Roger Grosse: https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
